In [ ]:
!pip -q install langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00


In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.315
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, anyio, async-timeout, dataclasses-json, jsonpatch, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


# Summarization

In [ ]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


### Setting up Summarization Chain

In [ ]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0)

In [ ]:
!ls -al

total 24
drwxr-xr-x 1 root root 4096 Oct 16 12:18 .
drwxr-xr-x 1 root root 4096 Oct 16 12:02 ..
drwxr-xr-x 4 root root 4096 Oct 12 13:25 .config
-rw-r--r-- 1 root root 6947 Oct 16 12:18 gpu_shortage
drwxr-xr-x 1 root root 4096 Oct 12 13:25 sample_data


In [ ]:
# load the doc
with open('gpu_shortage') as f:
    gpu_shortage_essay = f.read()

In [ ]:
len(gpu_shortage_essay)

6751

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
texts = text_splitter.split_text(gpu_shortage_essay)

In [ ]:
len(texts)

9

In [ ]:
from langchain.docstore.document import Document

docs = [Document(page_content=t) for t in texts[:4]]

In [ ]:
docs

[Document(page_content='As compute-hungry generative AI shows no signs of slowing down, which companies are getting access to Nvidia’s hard-to-come-by, ultra-expensive, high-performance computing H100 GPU for large language model (LLM) training is becoming the “top gossip” of Silicon Valley, according to Andrej Karpathy, former director of AI at Tesla and now at OpenAI.\n\nKarpathy’s comments come at a moment where issues related to GPU access are even being discussed in big tech annual reports: In Microsoft’s annual report released last week, the company emphasized to investors that GPUs are a “critical raw material for its fast-growing cloud business” and added language about GPUs to a “risk factor for outages that can arise if it can’t get the infrastructure it needs.”'),
 Document(page_content='Karpathy took to the social network X (formerly Twitter) to re-share a widely circulated blog post thought to be authored by a poster on Hacker News that speculates “the capacity of large sc

##  3 types of CombineDocuments Chains

[Taken from the LangChain Docs](https://langchain.readthedocs.io/en/latest/modules/indexes/combine_docs.html)

## Summarize Simple with map_reduce

In [ ]:
prompt_template = """Write a concise bullet point summary of the following:

{text}

CONSCISE SUMMARY IN BULLET POINTS:"""

BULLET_POINT_PROMPT = PromptTemplate(template=prompt_template,
                        input_variables=["text"])



### Stuffing
Stuffing is the simplest method, whereby you simply stuff all the related data into the prompt as context to pass to the language model. This is implemented in LangChain as the StuffDocumentsChain.

**Pros:** Only makes a single call to the LLM. When generating text, the LLM has access to all the data at once.

**Cons:** Most LLMs have a context length, and for large documents (or many documents) this will not work as it will result in a prompt larger than the context length.

The main downside of this method is that **it only works one smaller pieces of data.**  Once you are working with many pieces of data, this approach is no longer feasible. The next two approaches are designed to help deal with that.



In [ ]:
chain = load_summarize_chain(llm,
                             chain_type="stuff",
                             prompt=BULLET_POINT_PROMPT)

output_summary = chain.run(docs)

wrapped_text = textwrap.fill(output_summary,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)


- Nvidia's H100 GPU is in high demand for large language model (LLM) training
- Companies such as
OpenAI, Inflection, Meta, Azure, Google Cloud, AWS, Oracle, Lambda, CoreWeave, Anthropic, Helsing,
Mistral, Character, ByteDance, Baidu, Tencent, Jane Street, JP Morgan, Two Sigma, and Citadel are
all vying for access to the H100 GPU
- Demand for GPUs is likened to the television show 'Game of
Thrones'
- Radical Ventures has invested in CentML to optimize ML models and increase the supply of
chips in the market
- d-Matrix is building a platform to save money on inference by doing more
processing in the computer's memory, rather than on a GPU


### map_reduce summarization with custom prompt

In [ ]:
# with a custom prompt
prompt_template = """Write a concise summary of the following:

{text}

CONSCISE SUMMARY IN BULLET POINTS:"""

PROMPT = PromptTemplate(template=prompt_template,
                        input_variables=["text"])

## with intermediate steps
chain = load_summarize_chain(OpenAI(temperature=0),
                             chain_type="map_reduce",
                             return_intermediate_steps=True,
                             map_prompt=PROMPT,
                             combine_prompt=PROMPT)

output_summary = chain({"input_documents": docs}, return_only_outputs=True)
wrapped_text = textwrap.fill(output_summary['output_text'],
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)


• Nvidia's H100 GPU is becoming a hot topic in Silicon Valley due to its high-performance computing
capabilities.
• Andrej Karpathy, former director of AI at Tesla, has commented on the issue.
•
Microsoft's annual report has highlighted GPUs as a "critical raw material" for its cloud business.
• Estimates suggest that 432,000 H100s will be purchased, worth approximately $15 billion.
• Radical
Ventures has invested in CentML to optimize ML models and create more efficiency in the market.
•
d-Matrix is building a platform to save money on inference by doing more processing in the
computer's memory.


In [ ]:
wrapped_text = textwrap.fill(output_summary['intermediate_steps'][2],
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)


- David Katz, partner at Radical Ventures, compared the battle to get access to AI chips to the
television hit ‘Game of Thrones’
- He noted the “insatiable appetite for compute” needed to run
large models
- Estimates suggest that 432,000 H100s will be purchased, worth approximately $15
billion
- Chinese companies such as ByteDance, Baidu and Tencent are expected to purchase H800s
-
Financial companies such as Jane Street, JP Morgan, Two Sigma and Citadel are expected to purchase
hundreds to thousands of A/H100s


### With the 'refine' CombineDocument Chain

## Refine
This method involves **an initial prompt on the first chunk of data, generating some output. For the remaining documents, that output is passed in, along with the next document**, asking the LLM to refine the output based on the new document.

**Pros:** Can pull in more relevant context, and may be less lossy than MapReduceDocumentsChain.

**Cons:** Requires many more calls to the LLM than StuffDocumentsChain. The calls are also NOT independent, meaning they cannot be paralleled like MapReduceDocumentsChain. There is also some potential dependencies on the ordering of the documents.

In [ ]:
chain = load_summarize_chain(llm, chain_type="refine")

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

  Nvidia's H100 GPU, a hard-to-come-by and expensive high-performance computing device, is becoming
a hot topic in Silicon Valley. Andrej Karpathy, former director of AI at Tesla and now at OpenAI,
has commented on the issue, and Microsoft's annual report has also addressed the importance of GPUs
for its cloud business. Karpathy took to the social network X (formerly Twitter) to re-share a
widely circulated blog post thought to be authored by a poster on Hacker News that speculates “the
capacity of large scale H100 clusters at small and large cloud providers is running out,” and that
H100 demand will continue its trend till the end of 2024, at a minimum. The post estimates that
OpenAI might want 50,000 H100s, while Inflection wants 22,000, Meta “maybe 25k,” and “big clouds”
such as Azure, Google Cloud, AWS, and Oracle might want 30k each. Additionally, Lambda, CoreWeave,
Anthropic, Helsing, Mistral, and Character might want 10k each. Last year, Radical invested in
CentML, which optimiz